<a href="https://colab.research.google.com/github/tusharvatsa32/Language-Models/blob/main/Deep_Bow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/tusharvatsa32/nn4nlp-code.git

Cloning into 'nn4nlp-code'...
remote: Enumerating objects: 545, done.
remote: Total 545 (delta 0), reused 0 (delta 0), pack-reused 545
Receiving objects: 100% (545/545), 11.92 MiB | 19.18 MiB/s, done.
Resolving deltas: 100% (217/217), done.


In [2]:
pip install dynet

     |████████████████████████████████| 4.5MB 9.7MB/s 


In [5]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

# Read in the data
train = list(read_dataset("nn4nlp-code/data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("nn4nlp-code/data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [6]:
EMB_SIZE = 64
HID_SIZE = 64
HID_LAY = 2
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE)) # Word embeddings
W_h = [model.add_parameters((HID_SIZE, EMB_SIZE if lay == 0 else HID_SIZE)) for lay in range(HID_LAY)]
b_h = [model.add_parameters((HID_SIZE)) for lay in range(HID_LAY)]
W_sm = model.add_parameters((ntags, HID_SIZE))          # Softmax weights
b_sm = model.add_parameters((ntags))                    # Softmax bias

In [7]:
def calc_scores(words):
  dy.renew_cg()
  h = dy.esum([dy.lookup(W_emb, x) for x in words])
  for W_h_i, b_h_i in zip(W_h, b_h):
    h = dy.tanh( dy.parameter(W_h_i) * h + dy.parameter(b_h_i) )
  return dy.parameter(W_sm) * h + dy.parameter(b_sm)

In [8]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

The dy.parameter(...) call is now DEPRECATED.
        There is no longer need to explicitly add parameters to the computation graph.
        Any used parameter will be added automatically.
iter 0: train loss/sent=1.5123, time=0.85s
iter 0: test acc=0.3955
iter 1: train loss/sent=1.1781, time=0.79s
iter 1: test acc=0.4163
iter 2: train loss/sent=0.8860, time=0.80s
iter 2: test acc=0.4190
iter 3: train loss/sent=0.6168, time=0.79s
iter 3: test acc=0.4009
iter 4: train loss/sent=0.4294, time=0.79s
iter 4: test acc=0.3923
iter 5: train loss/sent=0.3035, time=0.80s
iter 5: test acc=0.3796
iter 6: train loss/sent=0.2197, time=0.79s
iter 6: test acc=0.3805
iter 7: train loss/sent=0.1650, time=0.79s
iter 7: test acc=0.3855
iter 8: train loss/sent=0.1288, time=0.81s
iter 8: test acc=0.3860
iter 9: train loss/sent=0.1087, time=0.80s
iter 9: test acc=0.3828
iter 10: train loss/sent=0.0881, time=0.83s
iter 10: test acc=0.3783
iter 11: train loss/sent=0.0761, time=0.79s
iter 11: test acc=0.3719
ite